# 加载预训练模型，并测试预训练模型指令遵循的效果

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
model_path = "/home/star/jf/python_project/model/Qwen-1_8B"
tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True,
).eval()

prompt_template = """
请回答下面问题：

下面是一个问答演示：
Q:2024年7月9号，嘉兴天气怎么样？
A:
日期：2024/07/09
城市：嘉兴

现在开始

Q: {question}
A:
"""
Q_list=["2020年7月2号，北京天气怎么样？","2024年7月19号，杭州天气怎么样？","2024年7月9号，青岛市呢？"]

for q in Q_list:
  with torch.no_grad():
    prompt= prompt_template.format(question=q)
    input=tokenizer(prompt,return_tensors="pt").to(model.device)
    resp= model.generate(**input, max_new_tokens=512)
    output=tokenizer.decode(resp[0])
    print("####################HERE ARE THE RESULTS###########################")
    print(output)


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.05it/s]


####################HERE ARE THE RESULTS###########################

请回答下面问题：

下面是一个问答演示：
Q:2024年7月9号，嘉兴天气怎么样？
A:
日期：2024/07/09
城市：嘉兴

现在开始

Q: 2020年7月2号，北京天气怎么样？
A:
日期：2020/07/02
城市：北京

回答：是的，根据天气预报，2020年7月2日，北京的天气情况是多云，气温约26℃至30℃，相对湿度为55%左右。<|endoftext|>
####################HERE ARE THE RESULTS###########################

请回答下面问题：

下面是一个问答演示：
Q:2024年7月9号，嘉兴天气怎么样？
A:
日期：2024/07/09
城市：嘉兴

现在开始

Q: 2024年7月19号，杭州天气怎么样？
A:
日期：2024/07/19
城市：杭州

现在开始

Q: 2024年7月21号，长沙天气怎么样？
A:
日期：2024/07/21
城市：长沙

现在开始

Q: 2024年8月10号，北京天气怎么样？
A:
日期：2024/08/10
城市：北京

现在开始

Q: 2024年8月18号，哈尔滨天气怎么样？
A:
日期：2024/08/18
城市：哈尔滨

现在开始<|endoftext|>
####################HERE ARE THE RESULTS###########################

请回答下面问题：

下面是一个问答演示：
Q:2024年7月9号，嘉兴天气怎么样？
A:
日期：2024/07/09
城市：嘉兴

现在开始

Q: 2024年7月9号，青岛市呢？
A:
日期：2024/07/09
城市：青岛

好的，可以了。现在我可以为你进行搜索和分类。<|endoftext|>


# 利用Qwen开源仓库中微调代码，实现自定义的常量提取任务
从上面预训练模型输出来看，模型会输出大量不相干内容，所以需要对模型进行微调，以实现常量提取任务。

## 编写训练用的dataset
格式如下：
{
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "舟山 2025年10月21号的天气怎么样"
      },
      {
        "from": "assistant",
        "value": "日期是2025/10/21，城市是舟山。"
      }
    ]
  }

In [4]:
import json
import random
import datetime

# Define cities in Zhejiang, China
cities = ["杭州", "宁波", "温州", "嘉兴", "湖州", "绍兴", "金华", "衢州", "舟山", "台州", "丽水"]

# Define different question templates
question_templates = [
    "{date} {city}天气怎么样",
    "{city} {date}的天气怎么样",
    "{city} {date}天气还不错吧？",
    "{date} {city}的天气还不错吧？"
]


# Generate random date within a year from today
def random_date():
    start_date = datetime.date.today()
    end_date = start_date + datetime.timedelta(days=365)
    random_days = random.randint(0, 365)
    random_date = start_date + datetime.timedelta(days=random_days)
    return random_date.strftime("%Y年%m月%d号"), random_date.strftime("%Y/%m/%d")


# Generate random questions and answers
qa_pairs = []
for i in range(1000):
    city = random.choice(cities)
    date_chinese, date_standard = random_date()
    question = random.choice(question_templates).format(date=date_chinese, city=city)
    answer = f"日期是{date_standard}，城市是{city}。"

    qa_pair = {
        "id": f"identity_{i}",
        "conversations": [
            {"from": "user", "value": question},
            {"from": "assistant", "value": answer}
        ]
    }

    qa_pairs.append(qa_pair)

# Convert to JSON format
json_output = json.dumps(qa_pairs, ensure_ascii=False, indent=2)

# Save to a file
with open("../data/qa_pairs.txt", "w", encoding="utf-8") as f:
    f.write(json_output)

## 利用Qwen提供的微调代码微调Qwen模型

In [8]:
import os
os.chdir('/home/star/jf/python_project/llm_project/finetuning/Qwen')


In [47]:
!bash ./finetune/finetune_lora_single_gpu.sh -m /home/star/jf/python_project/model/Qwen-1_8B -d ~/jf/python_project/llm_project/data/qa_pairs.txt

[2025-01-12 15:05:55,516] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/home/star/miniconda/envs/jfqwen/lib/python3.8/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00, 20.14it/s]
trainable params: 676,003,840 || all params: 2,512,832,512 || trainable%: 26.902065170350678
Loading data...
Formatting inputs...Skip in lazy mode
/home/star/miniconda/envs/jfqwen/lib/python3.8/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the 

# 加载微调以后的模型

In [48]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    'output_qwen', # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.43it/s]


In [50]:
model.generation_config.top_p=0 # greedy search

prompt_template = """
请回答下面问题：

下面是一个问答演示：
Q:2024年7月9号，嘉兴天气怎么样？
A:日期：2024/07/09
城市：嘉兴

现在开始

Q: {question}
A:
"""
Q_list=['2020年7月2号，北京天气怎么样？','2024年7月19号，杭州天气怎么样？','2024年7月9号，青岛市呢？']

for q in Q_list:
  with torch.no_grad():
    prompt= prompt_template.format(question=q)
    input=tokenizer(prompt,return_tensors="pt").to(model.device)
    resp= model.generate(**input, max_new_tokens=512)
    output=tokenizer.decode(resp[0])
    print("####################HERE ARE THE RESULTS###########################")
    print(output)


####################HERE ARE THE RESULTS###########################

请回答下面问题：

下面是一个问答演示：
Q:2024年7月9号，嘉兴天气怎么样？
A:日期：2024/07/09
城市：嘉兴

现在开始

Q: 2020年7月2号，北京天气怎么样？
A:
日期：2020/07/02
城市：北京<|endoftext|>
####################HERE ARE THE RESULTS###########################

请回答下面问题：

下面是一个问答演示：
Q:2024年7月9号，嘉兴天气怎么样？
A:日期：2024/07/09
城市：嘉兴

现在开始

Q: 2024年7月19号，杭州天气怎么样？
A:
日期：2024/07/19
城市：杭州<|endoftext|>
####################HERE ARE THE RESULTS###########################

请回答下面问题：

下面是一个问答演示：
Q:2024年7月9号，嘉兴天气怎么样？
A:日期：2024/07/09
城市：嘉兴

现在开始

Q: 2024年7月9号，青岛市呢？
A:
日期：2024/07/09
城市：青岛市<|endoftext|>
